# Topic 2, Part 2 - CPA on Hardware AES Implementation

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *By now you should have a pretty good understanding of how software implementations of AES are vulnerable to CPA attacks. You might be wondering: are hardware implementations of AES also vulnerable to CPA attacks?*

*In this lab, we'll perform a CPA attack on the hardware AES implementation in the STM32F415. We'll also introduce LASCAR for increased performance when analyzing large datasets.*

**LEARNING OUTCOMES:**
* Understanding how leakage differs between software AES and hardware AES implementations
* Using LASCAR for CPA attacks
* Identifying different leakage points

Capture traces as normal. We'll need to select the HWAES crypto target instead of TINYAES or MBEDTLS. Also we don't need to capture as many traces - the whole AES block will fit in less than 2000 traces. We'll also boost the gain a little bit - HWAES won't result in as big of power spikes:

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'
VERSION = 'HARDWARE'
SS_VER = 'SS_VER_2_1'

CRYPTO_TARGET = 'TINYAES128C'
allowable_exceptions = None


In [2]:
try:
    from lascar import *
except Exception as e:
    raise ImportError("You need to install lascar at https://github.com/Ledger-Donjon/lascar") from e

ImportError: You need to install lascar at https://github.com/Ledger-Donjon/lascar

In [3]:
CRYPTO_TARGET = 'HWAES' # overwrite auto inserted CRYPTO_TARGET

In [4]:
if VERSION == 'HARDWARE':
    
    #!/usr/bin/env python
    # coding: utf-8
    
    # # Topic 2, Part 2 - CPA on Hardware AES Implementation
    
    # ---
    # NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.
    # 
    # ---
    
    # Capture traces as normal. We'll need to select the HWAES crypto target instead of TINYAES or MBEDTLS. Also we don't need to capture as many traces - the whole AES block will fit in less than 2000 traces. We'll also boost the gain a little bit - HWAES won't result in as big of power spikes:
    
    # In[ ]:
    
    
    #SCOPETYPE = 'OPENADC'
    #PLATFORM = 'CW308_STM32F4'
    #CRYPTO_TARGET = 'HWAES'
    
    
    # In[ ]:
    
    
    
    #!/usr/bin/env python
    # coding: utf-8
    
    # In[ ]:
    
    
    import chipwhisperer as cw
    
    try:
        if not scope.connectStatus:
            scope.con()
    except NameError:
        scope = cw.scope(hw_location=(5, 7))
    
    try:
        if SS_VER == "SS_VER_2_1":
            target_type = cw.targets.SimpleSerial2
        elif SS_VER == "SS_VER_2_0":
            raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
        else:
            target_type = cw.targets.SimpleSerial
    except:
        SS_VER="SS_VER_1_1"
        target_type = cw.targets.SimpleSerial
    
    try:
        target = cw.target(scope, target_type)
    except:
        print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
        print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
        scope = cw.scope(hw_location=(5, 7))
        target = cw.target(scope, target_type)
    
    
    print("INFO: Found ChipWhisperer😍")
    
    
    # In[ ]:
    
    
    if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
        prog = cw.programmers.STM32FProgrammer
    elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        prog = cw.programmers.XMEGAProgrammer
    elif "neorv32" in PLATFORM.lower():
        prog = cw.programmers.NEORV32Programmer
    elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
        prog = cw.programmers.SAM4SProgrammer
    else:
        prog = None
    
    
    # In[ ]:
    
    
    import time
    time.sleep(0.05)
    scope.default_setup()
    
    def reset_target(scope):
        if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
            scope.io.pdic = 'low'
            time.sleep(0.1)
            scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
            time.sleep(0.1) #xmega needs more startup time
        elif "neorv32" in PLATFORM.lower():
            raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
        elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
            scope.io.nrst = 'low'
            time.sleep(0.25)
            scope.io.nrst = 'high_z'
            time.sleep(0.25)
        else:  
            scope.io.nrst = 'low'
            time.sleep(0.05)
            scope.io.nrst = 'high_z'
            time.sleep(0.05)
    
    

    
    
    # In[ ]:
    
    
    try:
        get_ipython().run_cell_magic('bash', '-s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"', 'cd ../../../firmware/mcu/simpleserial-aes\nmake PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3\n &> /tmp/tmp.txt')
    except:
        x=open("/tmp/tmp.txt").read(); print(x); raise OSError(x)

    
    
    # In[ ]:
    
    
    fw_path = '../../../firmware/mcu/simpleserial-aes/simpleserial-aes-{}.hex'.format(PLATFORM)
    cw.program_target(scope, prog, fw_path)
    
    
    # In[ ]:
    
    
    project = cw.create_project("traces/STM32F4_HW_AES.cwp", overwrite=True)
    
    
    # In[ ]:
    
    
    #Capture Traces
    from tqdm.notebook import trange, trange
    import numpy as np
    import time
    
    ktp = cw.ktp.Basic()
    
    traces = []
    N = 20000  # Number of traces
    scope.adc.samples=2000
    
    scope.gain.db = 32
    scope.glitch.arm_timing = "no_glitch"
    
    
    for i in trange(N, desc='Capturing traces'):
        key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    
        trace = cw.capture_trace(scope, target, text, key)
        if trace is None:
            continue
        project.traces.append(trace)
    
    print(scope.adc.trig_count)
    
    
    # In[ ]:
    
    
    scope.dis()
    target.dis()
    
    


INFO: Found ChipWhisperer😍


scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 98134                     to 5000                     
scope.adc.trig_count                     changed from 2402647621                to 2413801521               
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 25534286                  to 29538471                 
scope.clock.adc_rate                     changed from 25534286.0                to 29538471.0               
scope.clock.clkgen_

SS_VER set to SS_VER_2_1

SS_VER set to SS_VER_2_1

Blank crypto options, building for AES128

...

----------------
+--------------------------------------------------------
+ Built for platform CW30

8T: STM32F4 Target with:
+ CRYPTO_TARGET = HWAES
+ CRYPTO_OPTIONS = AES128C
+-----------------------

---------------------------------


Detected known STMF32: STM32F40xxx/41xxx
Extended erase (0x44), this can take ten seconds or more


FileNotFoundError: [Errno 2] No such file or directory: '../../../firmware/mcu/simpleserial-aes/simpleserial-aes-CW308_STM32F4.hex'

## Introducing LASCAR

With how many traces we're capturing, analyzing our traces will take a lot of time with ChipWhisperer - Analyzer wasn't designed for performance. It is for this reason that we will be using LASCAR, an open source side channel analysis library with a bigger emphasis on speed than ChipWhisperer Analyzer. Normally, it would take a bit of work to massage ChipWhisperer into the LASCAR format; however, ChipWhisperer has recently integrated some basic LASCAR support, making it easy to combine LASCAR and ChipWhisperer projects! Note that this support is a WIP and not offically documented - the interface can change at any time!

Basic setup is as follows:

In [5]:
import chipwhisperer.common.api.lascar as cw_lascar
from lascar import *
cw_container = cw_lascar.CWContainer(project, project.textouts, end=1000) #optional start and end args set start and end points for analysis
guess_range = range(256)

Unable to import LASCAR


NameError: name 'Container' is not defined

## Leakage Model

Thus far, we've been exclusively focusing on software AES. Here, each AES operation (shift rows, add round key, mix columns, etc) is implemented using one basic operation (XOR, reads/writes, multiplies, etc.) per clock cycle. With a hardware implementation, it's often possible to not only combine basic operations into a block that can run in a single clock cycle, but also combine multiple AES operations and run them in a single block! For example, the CW305 FPGA board can run each round of AES in a single clock cycle!

Because of this, running a CPA attack on hardware AES is much trickier than on software AES. In software, we found that it was easy to search for the outputs of the s-boxes because these values would need to be loaded from memory onto a high-capacitance data bus. This is not necessarily true for hardware AES, where the output of the s-boxes may be directly fed into the next stage of the algorithm. In general, we may need some more knowledge of the hardware implementation to successfully complete an attack. That being said, if we take a look at a block diagram of AES:

![](img/AES_Encryption.png)

the last round jumps out for a few reasons:

* It's not far removed from the ciphertext or the plaintext
* It's got an AddRoundKey and a SubBytes, meaning we get a nonlinear addition of the key between the ciphertext and the input of the round
* There's no Mix Columns

Let's make a guess at the implementation and say that it'll do the last round in a single clock cycle and store the input and output in the same memory block. Our reset assumption that allowed us to simply use the Hamming weight instead of the Hamming distance also probably won't be valid here. As such, let's use the Hamming distance between the output and the input of the last round.

ChipWhisperer now includes a few leakage models for use with LASCAR:

In [6]:
leakage = cw_lascar.lastround_HD_gen

NameError: name 'cw_lascar' is not defined

Then, we can actually run the analysis. It should chew through our 15k traces in only a minute or two!

In [7]:
cpa_engines = [CpaEngine("cpa_%02d" % i, leakage(i), guess_range) for i in range(16)]
session = Session(cw_container, engines=cpa_engines).run(batch_size=50)

NameError: name 'CpaEngine' is not defined

Let's print out our results and plot the correlation of our guesses:

In [8]:
import chipwhisperer.analyzer as cwa
plt = cw.plot([])
key_guess = []
last_round_key = cwa.aes_funcs.key_schedule_rounds(list(project.keys[0]),0,10)
#actual = last_round_key

for i in range(16):
    results = cpa_engines[i].finalize()
    guess = abs(results).max(1).argsort()[-1]
    guess2 = abs(results).max(1).argsort()[-2]
    print("Best Guess is {:02X} (Corr = {})".format(guess, abs(results).max()))
    actual = last_round_key[i]
    if guess != actual:
        plt *= cw.plot(results[guess]).opts(color='green')
    else:
        plt *= cw.plot(results[guess2]).opts(color='green')
    plt *= cw.plot(results[actual]).opts(color='red')
    key_guess.append(guess)

    
plt

NameError: name 'project' is not defined

ChipWhisperer also includes a class to interpret the results of the analysis:

In [9]:
import chipwhisperer.analyzer as cwa
last_round_key = cwa.aes_funcs.key_schedule_rounds(list(project.keys[0]),0,10)
disp = cw_lascar.LascarDisplay(cpa_engines, last_round_key)
disp.show_pge()

NameError: name 'project' is not defined

Interestingly, you should see that the attack has worked fairly well for most of the bytes. All of them, in fact, except bytes 0, 4, 8, and 12. Looking the correlation plot, you should see two large spikes instead of one like you might expect. Try focusing the attack on either one of these points by adjusting `start=` and `end=` when making the `cw_container` and try answering the following questions:

* Which spike was our expected leakage actually at (last round state diff)?
* How might you be able to tell that the attack failed for certain bytes at the incorrect leakage point?
* Why might this other spike be occuring?

In [10]:
scope.dis()
target.dis()

In [11]:
pge_avg = 0
for i in range(16):
    pge_avg += disp.results().calc_PGE(i)
    
pge_avg /= 16
assert pge_avg < 5, "PGE AVG: {}".format(pge_avg)

NameError: name 'disp' is not defined